In [1]:
import pandas as pd
import numpy as np
import requests
import bs4
import json
import os

In [2]:
from shapely.geometry import Point
import geopandas as gpd   

def to_mercator(lat, long):
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(distancia, distancia_2):
    return distancia.distance(distancia_2)

In [3]:
centros_culturales = requests.get('https://datos.madrid.es/egob/catalogo/200304-0-centros-culturales.json')
centros_culturales = centros_culturales.json()
df_centros_culturales = pd.json_normalize(centros_culturales['@graph'])
df_centros_culturales = pd.DataFrame(df_centros_culturales[["organization.organization-name" , "address.street-address" , "location.latitude" , "location.longitude"]])
df_centros_culturales = df_centros_culturales.assign(Tipo_Centro = "Centros culturales")
df_centros_culturales["distancia"] = df_centros_culturales.apply(lambda x : to_mercator(x["location.latitude"],x["location.longitude"]),axis=1)
#df_centros_culturales

In [4]:
museos = requests.get('https://datos.madrid.es/egob/catalogo/201132-0-museos.json')
museos = museos.json()
df_museos = pd.json_normalize(museos['@graph'])
df_museos = pd.DataFrame(df_museos[["organization.organization-name" , "address.street-address" , "location.latitude" , "location.longitude"]])
df_museos = df_museos.assign(Tipo_Centro = "Museos")
df_museos["distancia"] = df_museos.apply(lambda x : to_mercator(x["location.latitude"],x["location.longitude"]),axis=1)
#df_museos

In [5]:
df_museo_cc = pd.concat([df_centros_culturales , df_museos])
#df_museo_cc

In [6]:
df_bicimadstations = pd.read_json("../Database/bicimadstations.json")
lat =[float(index["geometry_coordinates"].split(",")[0].replace("[", "")) for row, index in df_bicimadstations.iterrows()]
lon = [float(index["geometry_coordinates"].split(",")[1].replace("]","")) for row, index in df_bicimadstations.iterrows()]
df_bicimadstations["latitud"] = lat
df_bicimadstations["longitud"] = lon
df_bicimadstations = pd.DataFrame(df_bicimadstations[["name" , "address" , "latitud" , "longitud"]])
df_bicimadstations["distancia_bicimad"] = df_bicimadstations.apply(lambda x : to_mercator(x["latitud"],x["longitud"]),axis=1)
#df_bicimadstations

In [7]:
df_museo_cc_bicimadstations = pd.merge(df_museo_cc, df_bicimadstations, how = 'cross')
df_museo_cc_bicimadstations["distancia_total"] = df_museo_cc_bicimadstations.apply(lambda x : distance_meters(x["distancia_bicimad"],x["distancia"]),axis=1)
#df_museo_cc_bicimadstations

In [ ]:
def Bicimad_mas_cercana():
    i = str(input("Introduzca lugar"))
    a = df_museo_cc_bicimadstations[df_museo_cc_bicimadstations["organization.organization-name"] == i]
    return a.sort_values(by = "distancia_total", ascending = True).head(1)
Bicimad_mas_cercana()
   

In [ ]:
def Bicimad_mas_cercanas_sitios_interes():
    i = str(input("Todas las estaciones mas cercana a cada sitio"))
    a = df_museo_cc_bicimadstations[df_museo_cc_bicimadstations["organization.organization-name"] == i]
    return a.sort_values(by = "distancia_total", ascending = True)
Bicimad_mas_cercanas_sitios_interes()

In [ ]:
def Bicimad_mas_cercanas_sitios_interes():
    
    return df_museo_cc_bicimadstations.sort_values(by = "distancia_total", ascending = True).groupby('organization.organization-name')['Tipo_Centro','address', 'distancia_total'].nth(0).drop(["distancia"], axis = "columns")
Bicimad_mas_cercanas_sitios_interes